In [1]:
from scipy.io import netcdf as nc
import numpy as np

In [2]:
def readnc(ncfile,varname):
        fid = nc.netcdf_file(ncfile, 'r')
        out = np.array(fid.variables[varname][:,:,:]).squeeze()
        fid.close()
        return out

In [3]:
def readnc_1d(ncfile,varname):
        fid = nc.netcdf_file(ncfile, 'r')
        out = np.array(fid.variables[varname][:]).squeeze()
        fid.close()
        return out

In [13]:
def write_list(ncfile,time,var,varname):
        # write a timeserie
        try:
                from netCDF4 import Dataset
        except:
                print('netCDF4 not available') ; exit()

        fid = Dataset(ncfile, 'w', format='NETCDF3_CLASSIC')
        fid.description = 'file created by Sommer A (anna.sommer.lab@gmail.com)'
        # dimensions
        fid.createDimension('axis', None)
        # variables
        times       = fid.createVariable('axis', 'f4', ('axis',))
        variable    = fid.createVariable(varname, 'f8', ('axis',))
        # data
        times[:]    = time
        variable[:] = var
        # close
        fid.close()
        return None

In [4]:
month_name = ['Jan','Jan','Jan','Jan','Jan','Jan','Feb','Feb','Feb','Feb','Feb','Feb','Mar','Mar','Mar','Mar','Mar','Mar','Apr','Apr','Apr','Apr','Apr','Apr','May','May','May','May','May','May','June','June','June','June','June','June','July','July','July','July','July','July','Aug','Aug','Aug','Aug','Aug','Aug','Aug','Sept','Sept','Sept','Sept','Sept','Sept','Oct','Oct','Oct','Oct','Oct','Oct','Nov','Nov','Nov','Nov','Nov','Nov','Dec','Dec','Dec','Dec','Dec','Dec',]

In [5]:
ro = 1025.
T0 = 273.15

A1 = -60.2409
A2 = 93.4517
A3 = 23.3585
B1 = 0.023517
B2 = -0.023656
B3 = 0.0047036

Sc_ref = 660.

C0 = 2073.1
C1 = -125.62
C2 = 3.6276
C3 = -0.043219

In [15]:
for time_year in np.arange(2008,2011,1):
    for num_file in np.arange(0,73,1):

        direct_lon = '/Volumes/LaCie/Data/Argo_NEMO_FFNN_Atl_20082010_nc/Data_predict_lon_'+str(time_year)+'_'+str(num_file)+'.nc'
        direct_lat = '/Volumes/LaCie/Data/Argo_NEMO_FFNN_Atl_20082010_nc/Data_predict_lat_'+str(time_year)+'_'+str(num_file)+'.nc'
        lon = readnc_1d(direct_lon,'lon_list')
        lat = readnc_1d(direct_lat,'lat_list')

        direct_atm = '/Volumes/LaCie/Data/Argo_NEMO_FFNN_Atl_20082010_nc/Data_predict_pCO2_atm_'+str(time_year)+'_'+str(num_file)+'.nc'
        pCO2_atm = readnc_1d(direct_atm,'pCO2_atm_list')

        direct1_SSS = '/Volumes/LaCie/Data/Argo_NEMO_FFNN_Atl_20082010_nc/Data_predict_SSS_'+str(time_year)+'_'+str(num_file)+'.nc'
        direct1_SST = '/Volumes/LaCie/Data/Argo_NEMO_FFNN_Atl_20082010_nc/Data_predict_SST_'+str(time_year)+'_'+str(num_file)+'.nc'
        salinity = readnc_1d(direct1_SSS,'SSS_list')
        temperature = readnc_1d(direct1_SST,'SST_list')
        
        L = 0.996 * np.exp(A1 + A2 * (100./(temperature + T0)) + A3 * np.log((temperature + T0)/100.) + salinity * (B1 + B2 * (temperature + T0)/100. + B3 * ((temperature + T0)/100.)**2))

        Sc = C0 + C1 * (temperature) + C2 * (temperature )**2 + C3 * (temperature)**3
        
        direct_wind_u = '/Volumes/LaCie/Data/Wind_NCEP_Interp_OSSE/WindInterp_uwnd_'+str(time_year)+'_'+str(num_file)+'.nc'
        wind_u = readnc_1d(direct_wind_u,'wind10')
        direct_wind_v = '/Volumes/LaCie/Data/Wind_NCEP_Interp_OSSE/WindInterp_vwnd_'+str(time_year)+'_'+str(num_file)+'.nc'
        wind_v = readnc_1d(direct_wind_v,'wind10')
        
        ampl = (wind_u**2 + wind_v**2)
        
        Gi = 16.5 / np.nanmean(np.nanmean(ampl) * np.sqrt(Sc_ref/Sc))
        print('Gi', Gi)

        kCO2 = Gi * ampl * np.sqrt(Sc_ref/Sc)
        kCO2 = kCO2 /100. * 365. * 24. 
        
        for numb_of_model in np.arange(0,4,1):
            direct_SOCAT = '/Volumes/LaCie/Data/Network_Design_SOCAT_20012010_nc/pCO2_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_test1_'+str(numb_of_model)+'.nc'
            pCO2_SOCAT = readnc_1d(direct_SOCAT,'pCO2')

            fCO2_SOCAT = kCO2 * ro * L * (pCO2_SOCAT - pCO2_atm) * 10**(-6)       
            
            time = 0.   
            write_list('/Volumes/LaCie/Data/Network_Design_SOCAT_20012010_nc/fCO2_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_'+str(numb_of_model)+'.nc',time,fCO2_SOCAT,'fCO2')
   
            direct_Argo = '/Volumes/LaCie/Data/Network_Design_Argo_20082010_nc/pCO2_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_test1_'+str(numb_of_model)+'.nc'
            pCO2_Argo = readnc_1d(direct_Argo,'pCO2')

            fCO2_Argo = kCO2 * ro * L * (pCO2_Argo - pCO2_atm) * 10**(-6)       
            
            time = 0.   
            write_list('/Volumes/LaCie/Data/Network_Design_Argo_20082010_nc/fCO2_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_'+str(numb_of_model)+'.nc',time,fCO2_Argo,'fCO2')

            direct_SOCATArgo = '/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc/pCO2_Argo0810_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_test1_'+str(numb_of_model)+'.nc'
            pCO2_SOCATArgo = readnc_1d(direct_SOCATArgo,'pCO2')

            fCO2_SOCATArgo = kCO2 * ro * L * (pCO2_SOCATArgo - pCO2_atm) * 10**(-6)       
            
            time = 0.   
            write_list('/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc/fCO2_Argo0810_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_'+str(numb_of_model)+'.nc',time,fCO2_SOCATArgo,'fCO2')

            direct_SOCATArgo25 = '/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc25/pCO2_Argo0810_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_test1_'+str(numb_of_model)+'.nc'
            pCO2_SOCATArgo25 = readnc_1d(direct_SOCATArgo25,'pCO2')

            fCO2_SOCATArgo25 = kCO2 * ro * L * (pCO2_SOCATArgo25 - pCO2_atm) * 10**(-6)       
            
            time = 0.   
            write_list('/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc25/fCO2_Argo0810_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_'+str(numb_of_model)+'.nc',time,fCO2_SOCATArgo25,'fCO2')

            direct_SOCATArgo10 = '/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc10/pCO2_Argo0810_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_test1_'+str(numb_of_model)+'.nc'
            pCO2_SOCATArgo10 = readnc_1d(direct_SOCATArgo10,'pCO2')

            fCO2_SOCATArgo10 = kCO2 * ro * L * (pCO2_SOCATArgo10 - pCO2_atm) * 10**(-6)       
            
            time = 0.   
            write_list('/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc10/fCO2_Argo0810_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_'+str(numb_of_model)+'.nc',time,fCO2_SOCATArgo10,'fCO2')

            
            
            
            direct_SOCATArgoS = '/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc/pCO2_ArgoS0810_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_test1_'+str(numb_of_model)+'.nc'
            pCO2_SOCATArgoS = readnc_1d(direct_SOCATArgoS,'pCO2')

            fCO2_SOCATArgoS = kCO2 * ro * L * (pCO2_SOCATArgoS - pCO2_atm) * 10**(-6)       
            
            time = 0.   
            write_list('/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc/fCO2_ArgoS0810_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_'+str(numb_of_model)+'.nc',time,fCO2_SOCATArgoS,'fCO2')

            direct_SOCATArgoS25 = '/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc25/pCO2_ArgoS0810_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_test1_'+str(numb_of_model)+'.nc'
            pCO2_SOCATArgoS25 = readnc_1d(direct_SOCATArgoS25,'pCO2')

            fCO2_SOCATArgoS25 = kCO2 * ro * L * (pCO2_SOCATArgoS25 - pCO2_atm) * 10**(-6)       
            
            time = 0.   
            write_list('/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc25/fCO2_ArgoS0810_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_'+str(numb_of_model)+'.nc',time,fCO2_SOCATArgoS25,'fCO2')

            direct_SOCATArgoS10 = '/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc10/pCO2_ArgoS0810_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_test1_'+str(numb_of_model)+'.nc'
            pCO2_SOCATArgoS10 = readnc_1d(direct_SOCATArgoS10,'pCO2')

            fCO2_SOCATArgoS10 = kCO2 * ro * L * (pCO2_SOCATArgoS10 - pCO2_atm) * 10**(-6)       
            
            time = 0.   
            write_list('/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc10/fCO2_ArgoS0810_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_'+str(numb_of_model)+'.nc',time,fCO2_SOCATArgoS10,'fCO2')

            
            
            

            
            direct_SOCATArgoS_M = '/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc/pCO2_ArgoS0810_M_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_test1_'+str(numb_of_model)+'.nc'
            pCO2_SOCATArgoS_M = readnc_1d(direct_SOCATArgoS_M,'pCO2')

            fCO2_SOCATArgoS_M = kCO2 * ro * L * (pCO2_SOCATArgoS_M - pCO2_atm) * 10**(-6)       
            
            time = 0.   
            write_list('/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc/fCO2_ArgoS0810_M_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_'+str(numb_of_model)+'.nc',time,fCO2_SOCATArgoS_M,'fCO2')

            direct_SOCATArgoS25_M = '/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc25/pCO2_ArgoS0810_M_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_test1_'+str(numb_of_model)+'.nc'
            pCO2_SOCATArgoS25_M = readnc_1d(direct_SOCATArgoS25_M,'pCO2')

            fCO2_SOCATArgoS25_M = kCO2 * ro * L * (pCO2_SOCATArgoS25_M - pCO2_atm) * 10**(-6)       
            
            time = 0.   
            write_list('/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc25/fCO2_ArgoS0810_M_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_'+str(numb_of_model)+'.nc',time,fCO2_SOCATArgoS25_M,'fCO2')

            direct_SOCATArgoS10_M = '/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc10/pCO2_ArgoS0810_M_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_test1_'+str(numb_of_model)+'.nc'
            pCO2_SOCATArgoS10_M = readnc_1d(direct_SOCATArgoS10_M,'pCO2')

            fCO2_SOCATArgoS10_M = kCO2 * ro * L * (pCO2_SOCATArgoS10_M - pCO2_atm) * 10**(-6)       
            
            time = 0.   
            write_list('/Volumes/LaCie/Data/Network_Design_SOCATArgo_20012010_nc10/fCO2_ArgoS0810_M_reconstr_'+month_name[num_file]+'_'+str(time_year)+str(num_file)+'_'+str(numb_of_model)+'.nc',time,fCO2_SOCATArgoS10_M,'fCO2')
            

Gi 0.5213717476643286
Gi 0.5767455575108202
Gi 0.5050461617579068
Gi 0.6442499514172442
Gi 0.5002097382798503
Gi 0.496543863109782
Gi 0.6039772996952053
Gi 0.5518187282984931
Gi 0.5551047118654604
Gi 0.533458126221074
Gi 0.4576245267815317
Gi 0.5355849646250799
Gi 0.4996771250680435
Gi 0.4818512617296934
Gi 0.6537103875242647
Gi 0.5700043359972182
Gi 0.5632501310377325
Gi 0.5048838226661932
Gi 0.5494288953874205
Gi 0.5612433742701184
Gi 0.6335466178281957
Gi 0.5857976995105078
Gi 0.545872006579803
Gi 0.5211359782173101
Gi 0.6508073425841453
Gi 0.627493870634975
Gi 0.8647852493313017
Gi 0.5637646826369528
Gi 0.6840899258251624
Gi 0.5619783066701775
Gi 0.5221173422590679
Gi 0.5639284250423078
Gi 0.5719460488782913
Gi 0.6327930108943512
Gi 0.7270494778701139
Gi 0.575719343878287
Gi 0.5376487136116546
Gi 0.5545799300324228
Gi 0.544183730170322
Gi 0.7060225348376549
Gi 0.5632520579218293
Gi 0.536391205523691
Gi 0.7333539176239509
Gi 0.6780289212037045
Gi 0.6169243575710736
Gi 0.406488645321